In [2]:
import math
import itertools
import pandas as pd
import numpy as np
import random

# 1. CSV 불러오기
df = pd.read_csv("/content/random_coordinates.csv")

# 2. 전처리
def natural_key(s): return int(''.join(filter(str.isdigit, s)))
label_groups = {}
prerequisites = {}

for label in df["Label"].unique():
    items = sorted(df[df["Label"] == label]["ID"].tolist(), key=natural_key)
    label_groups[label] = items
    for i in range(1, len(items)):
        prerequisites[items[i]] = items[i - 1]

label_index = {label: {item: i for i, item in enumerate(ids)} for label, ids in label_groups.items()}
coords = dict(zip(df["ID"], zip(df["X"], df["Y"])))
all_items = set(df["ID"])

# 3. 거리 계산
def full_cycle_distance(order, base=(50, 50)):
    total = 0
    cur = base
    for item in order:
        total += math.dist(cur, coords[item])
        cur = coords[item]
    total += math.dist(cur, base)
    return total

# 4. 현재 보낸 상태 추적
def get_sent_indices(sent_path):
    state = {label: -1 for label in ['A', 'B', 'C']}
    for item in sent_path:
        label = item[0]
        idx = label_index[label][item]
        state[label] = max(state[label], idx)
    return state

# 5. DFS 기반 후보 생성
def generate_pick_lists_dfs_fixed_length(sent_path, fixed_length=4):
    pick_lists = []

    def dfs(path, sent_state):
        if len(path) == fixed_length:
            pick_lists.append(tuple(path))
            return
        for label in ['A', 'B', 'C']:
            next_idx = sent_state[label] + 1
            items = label_groups[label]
            if 0 <= next_idx < len(items):
                item = items[next_idx]
                prereq = prerequisites.get(item)
                if prereq is None or prereq in sent_path or prereq in path:
                    new_state = sent_state.copy()
                    new_state[label] += 1
                    dfs(path + [item], new_state)

    sent_state = get_sent_indices(sent_path) if sent_path else {'A': -1, 'B': -1, 'C': -1}
    dfs([], sent_state)
    return pick_lists

# 6. 소프트맥스 with temperature
def softmax(x, T=1.0):
    e_x = np.exp(-np.array(x) / T)
    return e_x / e_x.sum()

# 7. 최적화 설정
p_threshold = 0.95
num_simulations = 100
softmax_temperature = 0.7
lookahead_depth = 5

# 8. Top-p 기반 5단계 시뮬레이션 Lookahead
sent_path = []
pickup_log = []

while set(sent_path) != all_items:
    best_total_distance = float('inf')
    best_first_trip = None

    for _ in range(num_simulations):
        temp_sent = sent_path[:]
        temp_log = []

        for step in range(lookahead_depth):
            remaining = len(all_items - set(temp_sent))
            if remaining == 0:
                break
            cur_pick_len = min(4, remaining)
            picks = generate_pick_lists_dfs_fixed_length(temp_sent, fixed_length=cur_pick_len)

            candidates = []
            for pick in picks:
                for order in itertools.permutations(pick):
                    if all(
                        prerequisites.get(order[i]) is None or
                        prerequisites.get(order[i]) in temp_sent or
                        prerequisites.get(order[i]) in order[:i]
                        for i in range(len(order))
                    ):
                        candidates.append(order)

            if not candidates:
                break

            distances = [full_cycle_distance(c) for c in candidates]
            probs = softmax(distances, T=softmax_temperature)
            sorted_idx = np.argsort(probs)[::-1]
            cum_prob = np.cumsum([probs[i] for i in sorted_idx])
            cutoff = np.where(cum_prob >= p_threshold)[0][0] + 1
            top_p_idx = sorted_idx[:cutoff]

            chosen_idx = np.random.choice(top_p_idx, p=probs[top_p_idx] / probs[top_p_idx].sum())
            chosen = candidates[chosen_idx]

            temp_sent.extend(chosen)
            temp_log.append((chosen, round(full_cycle_distance(chosen), 2)))

        total_distance = sum(d for _, d in temp_log)
        if total_distance < best_total_distance and temp_log:
            best_total_distance = total_distance
            best_first_trip = temp_log[0][0]

    # 실제 적용
    if best_first_trip:
        sent_path.extend(best_first_trip)
        pickup_log.append({
            "pickup_sequence": best_first_trip,
            "trip_distance": round(full_cycle_distance(best_first_trip), 2)
        })
    else:
        break

# 9. 결과 출력
df_log = pd.DataFrame(pickup_log)
print(df_log)
print("\n✅ 총 이동 거리 합계:", round(df_log["trip_distance"].sum(), 2))


           pickup_sequence  trip_distance
0         (B1, A1, A2, B2)         209.44
1         (B3, C1, B4, A3)         136.57
2         (A4, A5, B5, C2)         193.14
3         (B6, A6, B7, C3)         160.00
4         (C4, A7, C5, C6)         113.14
..                     ...            ...
70    (A86, A87, A88, A89)         273.14
71    (A90, C96, A91, A92)         193.14
72    (A93, A94, A95, A96)         113.14
73    (C97, A97, C98, A98)         216.57
74  (C99, C100, A99, A100)         209.44

[75 rows x 2 columns]

✅ 총 이동 거리 합계: 13125.18
